In [ ]:
# syft absolute
import syft as sy
from syft.service.code.user_code import UserCode
from syft.service.request.request import Request

In [ ]:
server_low = sy.orchestra.launch(
    name="bigquery-low",
    server_side_type="low",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port="auto",
)

server_high = sy.orchestra.launch(
    name="bigquery-high",
    server_side_type="high",
    dev_mode=True,
    n_consumers=1,
    create_producer=True,
    port="auto",
)

In [ ]:
low_client = server_low.login(email="info@openmined.org", password="changethis")
high_client = server_high.login(email="info@openmined.org", password="changethis")

In [ ]:
# # todo: this is way too noisy
# widget = sy.sync(from_client=low_client, to_client=high_client)

In [ ]:
# widget

In [ ]:
# sync the users new request

In [ ]:
# syft absolute
from syft.client.syncing import compare_clients

In [ ]:
# syft absolute
from syft.service.job.job_stash import Job
from syft.service.job.job_stash import JobStatus

In [ ]:
def is_job_to_sync(batch):
    if batch.status != "NEW":
        return False
    if not isinstance(batch.root.high_obj, Job):
        return False
    job = batch.root.high_obj
    return job.status in (JobStatus.ERRORED, JobStatus.COMPLETED)

In [ ]:
def sync_new_objects(
    from_client, to_client, dry_run: bool = True, private_data: bool = False
):
    sim = "Simulating " if dry_run else ""
    priv = "WITH PRIVATE DATA" if private_data else ""
    print(f"{sim}Syncing from {from_client.name} to {to_client.name} {priv}")
    changes = []
    diff = compare_clients(
        from_client=from_client, to_client=to_client, hide_usercode=False
    )
    if isinstance(diff, sy.SyftError):
        return diff

    for batch in diff.batches:
        try:
            if is_job_to_sync(batch) or batch.status == "NEW":
                w = batch.resolve(build_state=False)
                if private_data:
                    w.click_share_all_private_data()
                if not dry_run:
                    w.click_sync()
                change_text = f"Synced {batch.status} {batch.root_type.__name__}"
                if not dry_run:
                    changes.append(change_text)
                else:
                    print(f"Would have run: {change_text}")
        except Exception as e:
            print("sync_new_objects", e)
            raise e
    return changes

In [ ]:
sync_new_objects(low_client, high_client)

In [ ]:
result = sync_new_objects(low_client, high_client, dry_run=False, private_data=True)

In [ ]:
assert "Synced NEW UserCode" in result
assert "Synced NEW Request" in result

In [ ]:
assert len(high_client.code.get_all()) == 2

In [ ]:
requests = high_client.requests
requests

In [ ]:
user_request = None

In [ ]:
for request in requests:
    if "large_sample" in getattr(
        getattr(request, "code", None), "service_func_name", None
    ):
        user_request = request

In [ ]:
assert user_request

In [ ]:
def execute_request(client, request) -> dict:
    if not isinstance(request, Request):
        return "This is not a request"

    code = request.code
    if not isinstance(code, UserCode):
        return "No usercode found"

    func_name = request.code.service_func_name
    api_func = getattr(client.code, func_name, None)
    if api_func is None:
        return "Code name was not found on the client."

    job = api_func(blocking=False)
    return job

In [ ]:
job = execute_request(high_client, user_request)

In [ ]:
job

In [ ]:
job.wait()

In [ ]:
# # todo: this is way too noisy
# widget = sy.sync(from_client=high_client, to_client=low_client)

In [ ]:
# widget

In [ ]:
sync_new_objects(high_client, low_client)

In [ ]:
result = sync_new_objects(high_client, low_client, dry_run=False, private_data=True)

In [ ]:
assert "Synced NEW Job" in result

In [ ]:
requests = low_client.requests
requests

In [ ]:
user_request = None

In [ ]:
for request in requests:
    if "large_sample" in getattr(
        getattr(request, "code", None), "service_func_name", None
    ):
        user_request = request

In [ ]:
user_request.status

In [ ]:
assert "approved" in str(user_request.status).lower()

In [ ]:
server_high.land()
server_low.land()